In [12]:
from praw import Reddit
from praw.models import Subreddit, Submission
import os
from dotenv import load_dotenv
from datetime import datetime, date, timedelta
from dotenv import dotenv_values
import datetime as dt
import pandas as pd


In [ ]:
# Load environment variables
config = dotenv_values(".env")

# Initialize Reddit API
reddit = Reddit(
    client_id=config["CLIENT_ID"],
    client_secret=config["CLIENT_SECRET"],
    redirect_uri=config["REDIRECT_URL"],
    user_agent=config["USER_AGENT"]
)

def fetch_posts_about_keyword(subreddit_name, keywords, limit):
    keyword_to_company = {
        "Uber": "Uber",
        "youtube": "YouTube",
        "meta": "Meta",
        "apple": "Apple",
        "nvidia": "NVIDIA",
        "microsoft": "Microsoft",
        "amazon": "Amazon",
        "Saudi Arabian Oil Co": "Saudi Arabian Oil Co",
        "intel": "Intel",
        "tesla": "Tesla",
        "MARA Holdings": "MARA Holdings"
    }
    
    subreddit = reddit.subreddit(subreddit_name)
    data = []
    for post in subreddit.hot(limit=limit):
        
        matching_keywords = [keyword for keyword in keywords if keyword.lower() in post.title.lower()]

        for keyword in keywords:
            if keyword.lower() in post.title.lower():
                post.comments.replace_more(limit=0)
                comments = post.comments.list()

                for comment in comments:
                    data.append({
                        'post_title': post.title,
                        'post_score': post.score,
                        'post_id': post.id,
                        'post_num_comments': post.num_comments,
                        'post_created': post.created,
                        'post_body': post.selftext,
                        'comment_id': comment.id,
                        'comment_body': comment.body,
                        'comment_score': comment.score,
                        'matched_keywords': ", ".join(matching_keywords)
                    })
                break 

    return data


subreddit_name = "stocks"  
keywords = ["Uber", "youtube", "meta", "apple", "nvidia", "microsoft", "amazon", "Saudi Arabian Oil Co", "intel", "tesla", "MARA Holdings"]
data = fetch_posts_about_keyword(subreddit_name, keywords, limit=500000)


df = pd.DataFrame(data)
df.to_csv("Reddit_comments.csv", index=False)